In [27]:
import random
import numpy as np

from init_model import generate_model

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

In [36]:
def get_bit_idx(num_nodes):
    L =  0 # genome length
    BITS_INDICES = np.empty((0, 2),dtype = np.int32)
    start = 0
    end = 0

    for x in num_nodes:
        end = end + sum(range(x))
        BITS_INDICES = np.vstack([BITS_INDICES, [start, end]])
        start = end
    L = end

    return L, BITS_INDICES

In [5]:
# Load the dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize the images to a range of 0 to 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reshape the data to include the channel dimension
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))


In [52]:
STAGES = np.array(["s1", "s2"]) # S
NUM_NODES = np.array([2, 3]) # K
NUM_LABELS = 10
FILTERS = np.array([32, 64])
L, BITS_INDICES = get_bit_idx(NUM_NODES)

POPULATION_SIZE = 5
NUM_GEN = 3

TRAINING_EPOCHS = 1

In [50]:
def evaluate_model(individual):
    model = generate_model(individual, STAGES, NUM_NODES, BITS_INDICES, FILTERS, NUM_LABELS)
    model.summary()

    model.fit(train_images, train_labels, epochs=TRAINING_EPOCHS, validation_data=(test_images, test_labels))
    
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    
    return test_acc, 


In [53]:
# Kiểm tra creator đã được khởi tạo với hai lớp FitnessMax và Individual hay chưa
if not hasattr(creator, "FitnessMax"):
    # Tạo ra một lớp FitnessMax mở rộng từ lớp Fitness (base class của DEAP),
    # được sử dụng để tối đa hoá đơn mục tiêu, giá trị Fitness càng cao càng tốt (weights=(1.0,))
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
if not hasattr(creator, "Individual"):
    # Tạo ra một lớp Individual có dạng là list, thuộc tính fitness là một instance của FitnessMax
    creator.create("Individual", list, fitness=creator.FitnessMax)


toolbox = base.Toolbox()

# Đăng ký một hàm có tên là binary được dùng để tạo ra một giá trị
#nhị phân theo phân phối bernoulli với xác suất 0.5 cho 1 và 0.5 cho 0
toolbox.register("binary", bernoulli.rvs, 0.5) 

# Đăng lý một hàm có tên là individual được gọi lặp lại để tạo ra một cá thể có L biến nhị phân,
# sử dụng lớp Individual được tạo ở trên 
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.binary, n = L)

# Đăng ký một hàm có tên là population được gọi lặp lại để tạo ra một quần thể các cá thể (list individual), 
# mỗi cá thể được tạo ra bằng hàm individual
toolbox.register("population", tools.initRepeat, list , toolbox.individual)

# Đăng ký một hàm lai ghép có tên là mate sử dụng hàm cxOrdered để hoán vị các biến
toolbox.register("mate", tools.cxOrdered)

# Đăng ký một hàm đột biến tên là mutate, sử dụng hàm mutShuffleIndexes để hoán đổi các index của các biến
# với xác suất đột biến là 0.8
toolbox.register("mutate", tools.mutShuffleIndexes, indpb = 0.8)

# Đăng ký một hàm chọn lọc với tên là select, sử dụng chiến lược chọn lọc roulette wheel
toolbox.register("select", tools.selRoulette)

# Đăng ký một hàm đánh giá tên là evaluate sử dụng hàm evaluateModel được định nghĩa ở trên
toolbox.register("evaluate", evaluate_model)

# Tạo ra một population với số lượng các cá thể là n
popl = toolbox.population(n = POPULATION_SIZE)

# Sử dụng simple evolutionary algorithm được cung cấp với DEAP
# với popl được tạo ra ở trên
# xác suất lai ghép là 0.4
# xác suất đột biến là 0.05
# số lượng thế hệ
# và in ra thông tin của quá trình thực thi
result = algorithms.eaSimple(popl, toolbox, cxpb = 0.4, mutpb = 0.05, ngen = NUM_GEN, verbose = True)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_tensor        │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s1_input (Conv2D)   │ (None, 28, 28,    │        832 │ input_tensor[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s1_input_bn         │ (None, 28, 28,    │        128 │ s1_input[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 14, 14,    │          0 │ s1_input_bn[0][0] │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_input (Conv2D)   │ (None, 14, 14,    │     51,264 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_input_bn         │ (None, 14, 14,    │        256 │ s2_input[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_1_conv (Conv2D)  │ (None, 14, 14,    │    102,464 │ s2_input_bn[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_2_conv (Conv2D)  │ (None, 14, 14,    │    102,464 │ s2_input_bn[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_1_conv_bn        │ (None, 14, 14,    │        256 │ s2_1_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_2_conv_bn        │ (None, 14, 14,    │        256 │ s2_2_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_1-s2_2 (Add)     │ (None, 14, 14,    │          0 │ s2_1_conv_bn[0][… │
│                     │ 64)               │            │ s2_2_conv_bn[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_3_conv (Conv2D)  │ (None, 14, 14,    │    102,464 │ s2_1-s2_2[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_3_conv_bn        │ (None, 14, 14,    │        256 │ s2_3_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_output (Conv2D)  │ (None, 14, 14,    │    102,464 │ s2_3_conv_bn[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s2_output_bn        │ (None, 14, 14,    │        256 │ s2_output[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 7, 7, 64)  │          0 │ s2_output_bn[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3136)      │          0 │ max_pooling2d_1[

 Total params: 664,778 (2.54 MB)

 Trainable params: 664,074 (2.53 MB)

 Non-trainable params: 704 (2.75 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_tensor (InputLayer)       │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s1_input (Conv2D)               │ (None, 28, 28, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s1_input_bn                     │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s2_input (Conv2D)               │ (None, 14, 14, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s2_input_bn                     │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s2_1_conv (Conv2D)              │ (None, 14, 14, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s2_1_conv_bn                    │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s2_2_conv (Conv2D)              │ (None, 14, 14, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s2_2_conv_bn                    │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s2_output (Conv2D)              │ (None, 14, 14, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ s2_output_bn                    │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 562,058 (2.14 MB)

 Trainable params: 561,482 (2.14 MB)

 Non-trainable params: 576 (2.25 KB)

 448/1875 ━━━━━━━━━━━━━━━━━━━━ 6:56 292ms/step - accuracy: 0.5339 - loss: 2.2622

In [ ]:
# # print top-3 optimal solutions 
# best_individuals = tools.selBest(popl, k = 3)
# for bi in best_individuals:
#     print(bi)